In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F


# model's hyperparameters
batch_size = 16
block_size = 32
max_iters = 6150
eval_interval = 120
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 64
n_head = 4
n_layer = 4
dropout = 0.0 # can add later
# ----------------
# ----------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('tiny_shakespeare.txt', 'r', encoding='utf-8') as f:
  text = f.read()

# all the unique characters that occur in this text:
uchars = sorted(list(set(text)))
vocab_size = len(uchars)
# create mapping (between) characters to integers
stoi = { ch:i for i,ch in enumerate(uchars) }
itos = { i:ch for i,ch in enumerate(uchars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string as input, output a list of integers
decode = lambda l: "".join([itos[i] for i in l]) # decoder : input is the list of integers, outputs a string (back)



# Train and test sets/splits:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.88*len(data))
train_data = data[:n]
val_data = data[n:]

# loading data
def get_batch(split):
  data = train_data if split == 'train' else val_data
  idx = torch.randint(len(data) - block_size,  (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  x, y = x.to(device), y.to(device)
  return x, y


@torch.no_grad() # this should say: "we don't intend to use backward() meaning no backprop (backpropagation) so no retaining of gradients values for model parameters needed
def estimate_loss():
  out = {}
  model.eval() 
  
  for split in ['train', 'val']:
      losses = torch.zeros(eval_iters)
      for k in range(eval_iters):
          X, Y = get_batch(split)
          logits, loss = model(X, Y)
          losses[k] = loss.item()
      out[split] = losses.mean()
  model.train()
  return out


class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  
  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)   # (B,T,C)
    q = self.query(x) # (B,T,C)

    # "affinities" or attention scores computation:
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B,T,T)
    wei = self.dropout(wei)

    # perform the weighted aggregation of the values
    v = self.value(x) # (B,T,C)
    out = wei @ v # (B,T,T) @ (B,T,C) --> (B, T, C)
    return out




class MultiHeadAttention(nn.Module):
  """
  Multiple heads of self-attention in parallel
  """

  def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

  def forward(self, x):
      out = torch.cat([h(x) for h in self.heads], dim=-1)
      out = self.dropout(self.proj(out))
      return out



class FeedForward(nn.Module):
  """
  simple linear layer followed by a non-linearity 
  """

  def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    
  def forward(self, x):
        return self.net(x)



class Block(nn.Module):
  """ 
  This is a Transformer block: communication followed by computation 
  
  """

  def __init__(self, n_embed, n_head):
        # n_embed: embedding dimension, n_head: the number of heads (that we would like to use)
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)


  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x


# Very simple bigram (language) model (We can go for a trigram model or more, from here) This is just to simplify the process as much as possible for now:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # lookup table, where each token directly reads off the logits for the next token, from it
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # last norm layer (LayerNorm)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets : tensor of integers, they are both (B,T)
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C) || device in case of converting a CPU Tensor with pinned memory to a CUDA Tensor !
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size) not necessarily the same as C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 


model = BigramLanguageModel()

m = model.to(device)
# Display/print the number of parameters of our model here:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters') # numel === number elements : number of elements (in total) in the tensor !!

#Optimizer (PyTorch)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  # evaluating the loss on our sets for train and val, sometimes
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"Step {iter}: training loss {losses['train']:.4f}, validation loss {losses['val']:.4f}")


  # a sampled batch of data:
  xb, yb = get_batch('train')
  #evaluation of the loss:
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()




# Using the compiled + trained model to generate max_new_tokens = 1337 :
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode((m.generate(context, max_new_tokens=1337))[0].tolist()))

0.209729 M parameters
Step 0: training loss 4.4077, validation loss 4.4038
Step 120: training loss 2.6161, validation loss 2.6279
Step 240: training loss 2.4616, validation loss 2.4683
Step 360: training loss 2.3653, validation loss 2.3953
Step 480: training loss 2.2997, validation loss 2.3197
Step 600: training loss 2.2396, validation loss 2.2600
Step 720: training loss 2.1870, validation loss 2.2069
Step 840: training loss 2.1378, validation loss 2.1784
Step 960: training loss 2.1070, validation loss 2.1571
Step 1080: training loss 2.0729, validation loss 2.1246
Step 1200: training loss 2.0247, validation loss 2.0904
Step 1320: training loss 2.0079, validation loss 2.0815
Step 1440: training loss 1.9863, validation loss 2.0630
Step 1560: training loss 1.9639, validation loss 2.0473
Step 1680: training loss 1.9370, validation loss 2.0263
Step 1800: training loss 1.9216, validation loss 2.0373
Step 1920: training loss 1.8923, validation loss 2.0128
Step 2040: training loss 1.8817, vali